### Automated Milling Workflow

In [ ]:
%load_ext autoreload
%autoreload 2

from odemis.acq.millmng import mill_patterns, MillingTaskSettings
from odemis.util.filename import make_unique_name
from pprint import pprint
from odemis import model
from odemis.acq.acqmng import acquire

from odemis.acq.stream import LiveStream, SEMStream, FIBStream, StaticSEMStream, FluoStream
import os
from pprint import pprint

In [ ]:
# connect to hardware

# stage
stage = model.getComponent(role="stage-bare")
print(f"Stage Position: {stage.position.value}")

# setup electron beam, det
electron_beam = model.getComponent(role="e-beam")
electron_det = model.getComponent(role="se-detector")
electron_focus = model.getComponent(role="ebeam-focus")

hwemtvas = set()
hwdetvas = set()

hwemt_vanames = ("beamCurrent", "accelVoltage", "resolution", "dwellTime", "horizontalFoV")
hwdet_vanames = ("brightness", "contrast", "detector_mode", "detector_type")
for vaname in model.getVAs(electron_beam):
    if vaname in hwemt_vanames:
        hwemtvas.add(vaname)
for vaname in model.getVAs(electron_det):
    if vaname in hwdet_vanames:
        hwdetvas.add(vaname)

sem_stream = SEMStream(
    name="SEM",
    detector=electron_det,
    dataflow=electron_det.data,
    emitter=electron_beam,
    focuser=electron_focus,
    hwemtvas=hwemtvas,
    hwdetvas=hwdetvas,
    blanker=None)

# setup ion beam, det
ion_beam = model.getComponent(role="ion-beam")
ion_det = model.getComponent(role="se-detector-ion")
ion_focus = model.getComponent(role="ion-focus")

hwemtvas = set()
hwdetvas = set()
# hwemt_vanames = ("beamCurrent", "accelVoltage", "resolution", "dwellTime", "horizontalFoV")
# hwdet_vanames = ("brightness", "contrast", "detector_mode", "detector_type")
for vaname in model.getVAs(ion_beam):
    if vaname in hwemt_vanames:
        hwemtvas.add(vaname)
for vaname in model.getVAs(ion_det):
    if vaname in hwdet_vanames:
        hwdetvas.add(vaname)

fib_stream = FIBStream(
    name="FIB",
    detector=ion_det,
    dataflow=ion_det.data,
    emitter=ion_beam,
    focuser=ion_focus,
    hwemtvas=hwemtvas,
    hwdetvas=hwdetvas,
)

print(f"SEM: {sem_stream}")
print(f"FIB: {fib_stream}")

# acquisitionStreams for FLM

ccd = model.getComponent(role="ccd")
light = model.getComponent(role="light")
focus = model.getComponent(role="focus")
em_filter = model.getComponent(role="filter")
fm_stream = FluoStream(
    name="FM",
    detector=ccd,
    dataflow=ccd.data,
    emitter=light,
    em_filter=em_filter,
    focuser=focus,
    # opm=self._main_data_model.opm,
    # detvas={"exposureTime"},
)

In [ ]:
f = acquire([sem_stream, fib_stream])

data, err  = f.result()

sem_image, fib_image  = data

%matplotlib inline
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(sem_image, cmap="gray")
ax[1].imshow(fib_image, cmap="gray")
# ax[2].imshow(fm_image, cmap="gray")
plt.show()


In [ ]:
f = acquire([fm_stream])
data, err  = f.result()

fm_image = data

%matplotlib inline
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(fm_image[0], cmap="gray")
plt.show()


In [ ]:
# for task_name, task in milling_tasks.items():
#     print(f"Task: {task_name}")
#     p = task.generate()
#     pprint(p)

## Automated Milling


In [ ]:
from odemis.acq.milling.feature import CryoLamellaFeature, CryoLamellaProject, create_new_project, create_new_feature
from odemis.acq.milling.tasks import load_milling_tasks, MILLING_ROUGH, MILLING_FINE, MILL_POLISHING
from typing import Dict


from datetime import datetime 
timestamp = datetime.timestamp(datetime.now())
# format as YYYY-MM-DD_HH-MM-SS
timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d_%H-%M-%S')
PROJECT_NAME = f"milling-project-{timestamp}"
PROJECT_PATH = "/home/patrick/development/scratch/project/automated-milling/"
os.makedirs(PROJECT_PATH, exist_ok=True)

# load milling tasks
task_list = [MILLING_ROUGH, MILLING_FINE, MILL_POLISHING]
milling_tasks = load_milling_tasks(os.path.join(PROJECT_PATH, "milling_tasks.yaml"), task_list)

# create project
project = create_new_project(PROJECT_PATH, PROJECT_NAME)


# features
# feature = CryoLamellaFeature(

create_new_feature(name="Lamella-1",
    position={"x": 50e-6, "y": 25e-6, "z": 15e-6, "rx": 0, "rz": 0},
    milling_tasks=milling_tasks,
    project=project,
)

create_new_feature(name="Lamella-1",
    position={"x": 33e-6, "y": 100e-6, "z": 66e-6, "rx": 0, "rz": 0},
    milling_tasks=milling_tasks,
    project=project,
)

pprint(project.features)

# steps
# 1. create feature
# 2. select position
# 3. select milling tasks
# 4. select alignment
# ready to mill


# TODO: ref image data -> when to acquire reference image
# TODO: save directory -> project directory / feature directory



In [ ]:
print(project.features.keys())

In [ ]:

f = stage.moveAbs({"x": 0, "y": 0, "z": 0, "rx": 0, "rz": 0})
f.result()


In [ ]:
# context manager wrapper

# class Future:
#     # ... existing code ...

#     @contextmanager
#     def check_cancelled(self):
#         with self._task_lock:
#             if self._task_state == CANCELLED:
#                 raise CancelledError()
#             yield

In [ ]:
# loop through each position, mill lamella

# TODO: set reference image correctly
for k, f in project.features.items():
    f.reference_image = fib_image

import numpy as np
from odemis.acq.millmng import AutomatedMillingManager
from odemis.util import executeAsyncTask
from concurrent import futures


def run_automated_milling(stage, sem_stream, fib_stream, fm_stream, task_list, project) -> futures.Future:
    """
    Automatically mill and image a list of features.

    :return: ProgressiveFuture
    """
    # Create a progressive future with running sub future
    future = model.ProgressiveFuture()
    # create automated milling task
    amm = AutomatedMillingManager(
        future=future,
        stage=stage,
        sem_stream=sem_stream,
        fib_stream=fib_stream,
        fm_stream=fm_stream,
        task_list=task_list,
        project=project,
    )
    # add the ability of cancelling the future during execution
    future.task_canceller = amm.cancel

    # set the progress of the future
    total_duration = 100000 # TODO: estimated duration
    import time
    future.set_end_time(time.time() + total_duration)

    # assign the acquisition task to the future
    executeAsyncTask(future, amm.run)

    return future

f: model.ProgressiveFuture = run_automated_milling(stage=stage, 
                          sem_stream=sem_stream, 
                          fib_stream=fib_stream, 
                          fm_stream=fm_stream, 
                          task_list=task_list, #[MILL_POLISHING], 
                          project=project)

def on_milling_done(f):
    print("Milling Done")

def update_progress(future, start, end):
    if hasattr(future, "msg"):
        print(f"PROGRESS UPDATED {future.msg}, {start}, {end}")

f.add_update_callback(update_progress)
f.add_done_callback(on_milling_done)
f.result()


# OVERVIEW Acquisitions

In [ ]:
from odemis.acq.stitching import get_tiled_areas, get_zstack_levels, acquireTiledArea, acquireOverview, FocusingMethod, REGISTER_IDENTITY, WEAVER_MEAN
from odemis.util.filename import create_filename
from odemis.dataio import find_fittest_converter

sample_centers_raw = stage.getMetadata()[model.MD_SAMPLE_CENTERS]
sample_centers = {n: (p["x"], p["y"]) for n, p in sample_centers_raw.items()}

# move to base
# stage.moveAbs({"x": 0, "y": 0, "z": 0, "rx": 0, "rz": 0}).result()

electron_beam.horizontalFoV.value = 500e-6
acq_streams = [sem_stream]

print(sample_centers)
areas = get_tiled_areas(
    pos=stage.position.value,
    streams=acq_streams,
    tiles_nx=2,
    tiles_ny=2,
    overlap=0.1,
    tiling_rng={"x": [-1e-3, 1e-3], "y": [-1e-3, 1e-3]},
    selected_grids=["GRID 1", "GRID 2"],
    sample_centers=sample_centers,
    whole_grid=True,
)

print(areas)

# BLOCKED WHILE THE UI IS OPEN????

In [ ]:
import matplotlib.pyplot as plt
for da in data:

    plt.imshow(da, cmap="gray")
    plt.show()

In [ ]:
data = acq_future.result(1)  # timeout is just for safety

In [ ]:
pprint(data[0].metadata)

In [ ]:
fib_stream.focuser.axes["z"].range

In [ ]:
from odemis.util.dataio import open_acquisition
import matplotlib.pyplot as plt

image = open_acquisition("/home/patrick/development/scratch/acq/grids/20240611-183438-overview.ome.tiff")

plt.imshow(image[0], cmap="gray")
plt.show()

In [ ]:
from odemis import model

stage = model.getComponent(role="stage-bare")
stage.moveAbs({"x": 0, "y": 0, "z": 0, "rx": 0, "rz": 0}).result()

In [ ]:
# SAMPLE CENTERS is posture dependent
# therefore it should be converted to a posture dependent value

print(sample_centers)

In [ ]:
from odemis import model

stage = model.getComponent(role="stage-bare")
sample_centers_raw = stage.getMetadata()[model.MD_SAMPLE_CENTERS]
sample_centers = {n: (p["x"], p["y"]) for n, p in sample_centers_raw.items()}

# get sem orientation
stage_md = stage.getMetadata()
sem_pos_active = stage_md[model.MD_FAV_SEM_POS_ACTIVE]
fm_pos_active = stage_md[model.MD_FAV_FM_POS_ACTIVE]


# update each sample center

for pos in sample_centers_raw.values():
    pos.update(sem_pos_active)
    print(pos)

print("sem: ", sem_pos_active)
print("fm: ", fm_pos_active)
print(sample_centers_raw)

from odemis.acq.move import MicroscopePostureManager, MeteorTFS1PostureManager, SEM_IMAGING, FM_IMAGING, POSITION_NAMES

pm: MeteorTFS1PostureManager = MicroscopePostureManager(model.getMicroscope())

posture = POSITION_NAMES[pm.getCurrentPostureLabel(pos)]
print(posture)

fm_pos_grid_01 = pm._transformFromSEMToMeteor(sample_centers_raw["GRID 1"])
fm_pos_grid_02 = pm._transformFromSEMToMeteor(sample_centers_raw["GRID 2"])

print("FM Positions")
print(fm_pos_grid_01)
print(fm_pos_grid_02)


grid_centres = {
    SEM_IMAGING: {
        "POSITIONS": {
        "GRID 1": sample_centers_raw["GRID 1"],
        "GRID 2": sample_centers_raw["GRID 2"],
    },
    "RANGE": stage_md[model.MD_SEM_IMAGING_RANGE],
    "STREAM": sem_stream,
    "FOCUSER": electron_focus,
    "DETECTOR": electron_det,
},
FM_IMAGING: {
    "POSITIONS": {
        "GRID 1": fm_pos_grid_01,
        "GRID 2": fm_pos_grid_02,
    },
    "RANGE": stage_md[model.MD_FM_IMAGING_RANGE],
    "STREAM": fm_stream,
    "FOCUSER": focus,
    "DETECTOR": ccd,
}
}


In [ ]:
# stage.moveAbs(fm_pos_grid_02).result()

In [ ]:



stage = model.getComponent(role="stage-bare")



grid = "GRID 1"
posture = FM_IMAGING
grid_pos = grid_centres[posture]["POSITIONS"][grid]

stage.moveAbs(grid_pos).result() # use pm for safety

grid_positions = grid_centres[posture]["POSITIONS"].items()
grid_centre_tuple = {n: (p["x"], p["y"]) for n, p in grid_positions}
acq_streams = [grid_centres[posture]["STREAM"]]
tiling_rng =  grid_centres[posture]["RANGE"]

focuser = grid_centres[posture]["FOCUSER"]
detector = grid_centres[posture]["DETECTOR"]


# print details:
print(f"Grid: {grid}")
print(f"Posture: {POSITION_NAMES[posture]}")
print(f"Grid Position: {grid_pos}")
print(f"Grid Centre Tuple: {grid_centre_tuple}")
print(f"Streams: {acq_streams}")
print(f"Tiling Range: {tiling_rng}")
print(f"Focuser: {focuser}")
print(f"Detector: {detector}")



areas = get_tiled_areas(
    pos=grid_pos,
    streams=acq_streams,
    tiles_nx=2,
    tiles_ny=2,
    overlap=0.1,
    tiling_rng=tiling_rng,
    selected_grids=["GRID 1"], #, "GRID 2"],
    sample_centers=grid_centre_tuple,
    whole_grid=True,
)

print(areas)

# TODO: these areas need to be projected along the sample plane

# overview settings
focus_mtd = FocusingMethod.NONE
use_autofocus = False
overlap = 0.1

save_dir = os.path.join("/home/patrick/development/scratch", f"acq/grid-fm-frame")
os.makedirs(save_dir, exist_ok=True)
filename = create_filename(save_dir, "{datelng}-{timelng}-overview",
                                        ".ome.tiff")
print(f"Filename: {filename}")
assert filename.endswith(".ome.tiff")
dirname, basename = os.path.split(filename)
tiles_dir = os.path.join(dirname, basename[:-len(".ome.tiff")] + "-tiles")
filename_tiles = os.path.join(tiles_dir, basename)
os.makedirs(tiles_dir, exist_ok=True)

print(filename_tiles)
print(f"Acquiring ...")


stage_fm = model.getComponent(role="stage")
acq_future = acquireOverview(
                streams=acq_streams,              # restrict to a single stream if using autofocus (sem, fib)
                stage=stage_fm, 
                areas=areas, 
                focus=focuser,             # replace with sem-focus, fib-focus
                detector=focuser,            # replace with detector
                overlap=overlap, 
                settings_obs=None, 
                log_path=filename_tiles, 
                zlevels=None,
                registrar=REGISTER_IDENTITY, 
                weaver=WEAVER_MEAN, 
                focusing_method=FocusingMethod.NONE,
                use_autofocus=use_autofocus)

data = acq_future.result()

exporter = find_fittest_converter(filename)
exporter.export(filename, data)


In [ ]:
import matplotlib.pyplot as plt
for da in data:

    plt.imshow(da, cmap="gray")
    plt.show()

In [ ]:
da.metadata